# Выявление количества вхождений слова "степь" в тексте и частоты слова "степь" относительно длины текста

In [1]:
import nltk
import re
import pymorphy2
import pandas as pd
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
morph = pymorphy2.MorphAnalyzer()

In [2]:
# загрузка таблицы
texts = pd.read_csv("texts_orb.csv")
texts

,Автор,Название,Год,Файл
0,Аксаков С.Т.,Детские годы Багрова-внука,1858,Aksakov_Detskiye_gody.txt
1,Аксаков С.Т.,Семейная хроника,1856,Aksakov_Hronika.txt
2,Аксаков С.Т.,Записки ружейного охотника Оренбургской губернии,1852,Aksakov_Zapiski_ohotnika.txt
3,Аношкин М.П.,А как зовут дочь,1978,Anoshkin_Kak_zovut_doch.txt
4,Бадьева Э.А.,Портрет на солнечной стороне,1959,Badyieva_Portret_na_slonechnoy_storone.txt
5,Валеев Р.,Должно же быть в жизни что-то такое,1974,Valeev_Dolzhno_zhe_byt_v_zhizni_chto-to_takoye...
6,Гавриленко И.,Хозяин,1981,Gavrilenko_Hozyain.txt
7,Гусев-Оренбургский С.И.,Страна отцов,1905,Gusev_Strana_otcov.txt
8,Даль В.И.,Уральский казак,1843,Dal_Uralskiy_Kazak.txt
9,Короленко В.Г.,У казаков,1901,Korolenko_U_kazakov.txt


In [83]:
# with open("Краснов.txt", "r", encoding='utf-8') as open_file:
#     text = open_file.read() 

In [4]:
# Определение количества строк, чтобы достать названия файлов для дальнейшей работы
index = list(range(0,len(texts)))
# index

In [5]:
# Создание списка файлов из таблицы
files_orb = []
for i in index:
    f = texts.iloc [i]['Файл']
    files_orb.append(f)
# files_orb

In [6]:
# Создание списка текстов
texts_orb = [] # предобработанные тексты
for i in files_orb:
    with open(i, "r", encoding='utf-8') as open_file:
        text = open_file.read() # чтение файла
        text = text.lower() # приведение к нижнему регистру
        text = text.replace("\n", " ") # удаление лишних символов
        text = text.replace("\xa0", "") # удаление лишних символов
        text = text.replace("\t\t", "")
        text = text.replace("-то", " -то")
#         text = text.translate(str.maketrans("", "", string.punctuation)) # удаление пунктуации
        texts_orb.append(text)           

In [30]:
texts_orb[51]

'мы сидим с бабкой на глиняной завалинке. стоит осень, вторая после начала войны. луна, поднимаясь, становится все меньше, светлей и бело освещает пустынную улицу. бабка о чем-то думает и смотрит в степь. я шепотом твержу стихи и вдруг сбиваюсь. — баб, — говорю ей, — стих забыл. — ну, ступай к деду. за выгоном пашут на быках колхозное поле. это метрах в трехстах, но мне, пятилетнему, идти туда далеко и боязно. под сандалиями шуршат стебли лебеды, уже отшумевшей свое лето. часто оглядываюсь назад: светлые крыши, темная бабка на завалинке. впереди на лунном небе, там, где его край подходит к сумеркам земли, появляются и исчезают силуэты: спина быка, шапка пахаря и взмахнувшая кнутом рука. слышны протяжные крики: — цоб, цобе! у костра возле пашни стоит бабка щербуха и, смеясь, кричит: — трохим григорич, к тебе унучек. опять стих забыл. мимо по борозде проходит упряжка быков с плугом, которой правит дед. догоняю, берусь за ручку плуга, чтобы не отстать. прохладно пахнет свежей развороченно

In [6]:
texts_orb_p = [] #  список текстов без пунктуации
for i in texts_orb:
    i = i.translate(str.maketrans("", "", string.punctuation)) # удаление пунктуации
    texts_orb_p.append(i)
# texts_orb_p[0:1]

In [7]:
# Токенизация текстов и создание списка из списка токенов
texts_orb_words = []
for i in texts_orb_p:
    word = word_tokenize(i)
    texts_orb_words.append(word)

In [8]:
# Определение длины каждого текста в словах
length = []
for i in texts_orb_words:
    l = len(i)
    length.append(l)
# length

In [9]:
# Очистка списка токенов от стоп-слов и лемматизация
words_orb_final = []
for i in texts_orb_words:
    w = []
    for j in i:
        if not j in stopwords.words('russian'):
            j = morph.parse(j)[0].normal_form
            w.append(j)
    words_orb_final.append(w)

In [10]:
# Вычисление частотности слов и создание списка с частотами слов по каждому тексту
freq_orb = []
for i in words_orb_final:
    freq = nltk.FreqDist(i)
    freq_orb.append(freq)
#     z = freq.most_common(10)
#     print(z)
# freq_orb

In [11]:
freq_orb[9]

FreqDist({'казак': 344, 'это': 248, 'свой': 193, 'который': 179, 'степь': 156, 'говорить': 136, 'войско': 121, 'наш': 115, 'сказать': 115, 'казачий': 106, ...})

In [12]:
# Создание списков абсолютного количества слова "степь" и частоты слова "степь"
step_orb_num = []
step_orb_freq = []

for i in freq_orb:
    for j in i:
        if j == 'степь':
            step_orb_num.append(i[j]) # абсолютное количество
    if not 'степь' in i:
        step_orb_num.append(0)
    f = i.freq('степь')
    step_orb_freq.append(f) # частота
    
# print(step_orb_num)
# print(step_orb_freq)

In [13]:
len(step_orb_num)
# step_orb_num

53

In [14]:
len(step_orb_freq)

53

In [15]:
# Создание новых колонок в таблице
texts = texts.assign(a=length, b=step_orb_num, c=step_orb_freq)
texts.rename(columns = {'a':'Длина текста', 'b':'Количество слова "cтепь"', 'c':'Частота слова "степь"'}, inplace = True)
display(texts)

,Автор,Название,Год,Файл,Длина текста,"Количество слова ""cтепь""","Частота слова ""степь"""
0,Аксаков С.Т.,Детские годы Багрова-внука,1858,Aksakov_Detskiye_gody.txt,106272,8,0.000125
1,Аксаков С.Т.,Семейная хроника,1856,Aksakov_Hronika.txt,65808,15,0.000357
2,Аксаков С.Т.,Записки ружейного охотника Оренбургской губернии,1852,Aksakov_Zapiski_ohotnika.txt,89933,77,0.001320
3,Аношкин М.П.,А как зовут дочь,1978,Anoshkin_Kak_zovut_doch.txt,4670,0,0.000000
4,Бадьева Э.А.,Портрет на солнечной стороне,1959,Badyieva_Portret_na_slonechnoy_storone.txt,6831,16,0.003521
5,Валеев Р.,Должно же быть в жизни что-то такое,1974,Valeev_Dolzhno_zhe_byt_v_zhizni_chto-to_takoye...,5906,1,0.000263
6,Гавриленко И.,Хозяин,1981,Gavrilenko_Hozyain.txt,3471,5,0.002352
7,Гусев-Оренбургский С.И.,Страна отцов,1905,Gusev_Strana_otcov.txt,44841,5,0.000147
8,Даль В.И.,Уральский казак,1843,Dal_Uralskiy_Kazak.txt,5878,4,0.001056
9,Короленко В.Г.,У казаков,1901,Korolenko_U_kazakov.txt,39923,156,0.005862


In [17]:
texts.to_csv('frequency_orb.csv', encoding = "utf-8") 

In [10]:
texts2 = pd.read_csv("texts_others.csv")
texts2

,Автор,Название,Год,Файл
0,Гоголь Н.В.,Тарас Бульба,1835,Gogol_Taras_Bulba.txt
1,Тургенев И.С.,Записки охотника,1947-1951,Turgenev_Zapiski_ohotnika.txt
2,Чехов А.П.,Степь,1888,Chekhov_Step.txt
3,Шолохов М.А.,Поднятая целина,1930-1959,Sholokhov_Podnyataya_tselina.txt
4,Шолохов М.А.,Тихий Дон,1925-1940,Sholokhov_Tikhiy_Don.txt


In [11]:
index2 = list(range(0,len(texts2)))
# index2

In [12]:
files_others = []
for i in index2:
    f = texts2.iloc [i]['Файл']
    files_others.append(f)
files_others

['Gogol_Taras_Bulba.txt',
 'Turgenev_Zapiski_ohotnika.txt',
 'Chekhov_Step.txt',
 'Sholokhov_Podnyataya_tselina.txt',
 'Sholokhov_Tikhiy_Don.txt']

In [13]:
texts_others = [] # предобработанные тексты
for i in files_others:
    with open(i, "r", encoding='utf-8') as open_file:
        text = open_file.read() # чтение файла
        text = text.lower() # приведение к нижнему регистру
        text = text.replace("\n", " ") # удаление лишних символов
        text = text.replace("\xa0", "") # удаление лишних символов
        text = text.replace("\t\t", "")
        text = text.replace("-то", " -то")
#         text = text.translate(str.maketrans("", "", string.punctuation)) # удаление пунктуации
        texts_others.append(text)   

In [25]:
texts_others_p = [] #  список текстов без пунктуации
for i in texts_others:
    i = i.translate(str.maketrans("", "", string.punctuation)) # удаление пунктуации
    texts_others_p.append(i)
# texts_others_p[0:1]

In [26]:
texts_others_words = []
for i in texts_others_p:
    word = word_tokenize(i)
    texts_others_words.append(word)

In [27]:
length2 = []
for i in texts_others_words:
    l = len(i)
    length2.append(l)
# length2

[37883, 103950, 28170, 209758, 431040]

In [28]:
words_others_final = []
for i in texts_others_words:
    w = []
    for j in i:
        if not j in stopwords.words('russian'):
            j = morph.parse(j)[0].normal_form
            w.append(j)
    words_others_final.append(w)

In [29]:
freq_others = []
for i in words_others_final:
    freq = nltk.FreqDist(i)
    freq_others.append(freq)
#     z = freq.most_common(10)
#     print(z)
# freq_others

In [30]:
step_others_num = []
step_others_freq = []

for i in freq_others:
    for j in i:
        if j == 'степь':
            step_others_num.append(i[j]) # абсолютное количество
    if not 'степь' in i:
        step_others_num.append(0)
    f = i.freq('степь')
    step_others_freq.append(f) # частота
    
# print(step_others_num)
# print(step_others_freq)

In [36]:
print(step_others_num)
print(step_others_freq)

[16, 5, 35, 91, 215]
[0.0006734857094751021, 7.44845667977595e-05, 0.0020061905307806947, 0.0006737546644553693, 0.0007099458459912825]


In [35]:
# texts2

,Автор,Название,Год,Файл
0,Гоголь Н.В.,Тарас Бульба,1835,Gogol_Taras_Bulba.txt
1,Тургенев И.С.,Записки охотника,1947-1951,Turgenev_Zapiski_ohotnika.txt
2,Чехов А.П.,Степь,1888,Chekhov_Step.txt
3,Шолохов М.А.,Поднятая целина,1930-1959,Sholokhov_Podnyataya_tselina.txt
4,Шолохов М.А.,Тихий Дон,1925-1940,Sholokhov_Tikhiy_Don.txt


In [38]:
texts2 = texts2.assign(a=length2, b=step_others_num, c=step_others_freq)
texts2.rename(columns = {'a':'Длина текста', 'b':'Количество слова "cтепь"', 'c':'Частота слова "степь"'}, inplace = True)
display(texts2)

,Автор,Название,Год,Файл,Длина текста,"Количество слова ""cтепь""","Частота слова ""степь"""
0,Гоголь Н.В.,Тарас Бульба,1835,Gogol_Taras_Bulba.txt,37883,16,0.000673
1,Тургенев И.С.,Записки охотника,1947-1951,Turgenev_Zapiski_ohotnika.txt,103950,5,0.000074
2,Чехов А.П.,Степь,1888,Chekhov_Step.txt,28170,35,0.002006
3,Шолохов М.А.,Поднятая целина,1930-1959,Sholokhov_Podnyataya_tselina.txt,209758,91,0.000674
4,Шолохов М.А.,Тихий Дон,1925-1940,Sholokhov_Tikhiy_Don.txt,431040,215,0.000710


In [39]:
texts2.to_csv('frequency_others.csv', encoding = "utf-8") 

# Поиск коллокаций в выделенных предложениях

In [40]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [7]:
target_sent_orb = [] #список предложений со словом степь и его производными
for i in texts_orb:
    sentence = sent_tokenize(i)
    for j in sentence:
        result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', j)
        if not result == None:
            target_sent_orb.append(j) # finding sentences with a target word and its derivatives and adding target sentences into the list 
# target_sent_orb[0:20]

In [9]:
# запись списка в файл
# sent = open('sentences.txt', 'w', encoding = 'utf-8')
# sent.writelines(target_sent_orb)
# sent.close()

In [8]:
sents = pd.DataFrame(target_sent_orb)
sents.columns = ['Sentence']
sents

,Sentence
0,"отец думал, что мать побоится ночной сырости; ..."
1,для меня опять готовилось новое зрелище; отлож...
2,"небо сверкало звездами, воздух был наполнен бл..."
3,"я видел, будто сквозь сон, как он садился, как..."
4,"степь, то есть безлесная и волнообразная беско..."
...,...
939,"мы остановились на том самом месте, где вчера ..."
940,это было первое мною виденное степное укреплен...
941,"вид со степи на укрепление грустнее еще, нежел..."
942,первые листки свои из степи он еще кое-как раз...


In [13]:
sents.to_csv('sentences.csv', encoding = "utf-8") 

In [103]:
# target_sent_orb[0:20]

['отец думал, что мать побоится ночной сырости; но место было необыкновенно сухо, никаких болот, и даже лесу не находилось поблизости, потому что начиналась уже башкирская степь; даже влажности ночного воздуха не было слышно.',
 'для меня опять готовилось новое зрелище; отложили лошадей, хотели спутать и пустить в поле, но как степные травы погорели от солнца и завяли, то послали в деревню за свежим сеном и овсом и за всякими съестными припасами.',
 'небо сверкало звездами, воздух был наполнен благовонием от засыхающих степных трав, речка журчала в овраге, костер пылал и ярко освещал наших людей, которые сидели около котла с горячей кашицей, хлебали ее и весело разговаривали между собою; лошади, припущенные к овсу, также были освещены с одной стороны полосою света… «не пора ли спать тебе, сережа?» – сказал мой отец после долгого молчания; поцеловал меня, перекрестил и бережно, чтоб не разбудить мать, посадил в карету.',
 'я видел, будто сквозь сон, как он садился, как тронулась карета 

In [43]:
target_sent_orb_p = [] #  список предложений без пунктуации
for i in target_sent_orb:
    i = i.translate(str.maketrans("", "", string.punctuation)) # удаление пунктуации
    target_sent_orb_p.append(i)

In [44]:
sent_orb_words = [] # токенизация предложений со словом степь
for i in target_sent_orb_p:
    word = word_tokenize(i)
    sent_orb_words.append(word)
# sent_orb_words[0:5]

In [91]:
# Очистка списка токенов от стоп-слов и лемматизация
words_sent_orb_final = []
for i in sent_orb_words:
    w = []
    for j in i:
        if not j in stopwords.words('russian'):
#             j = morph.parse(j)[0].normal_form # строка не нужна, если не лемматизировать
            w.append(j)
    words_sent_orb_final.append(w)

In [95]:
# words_sent_orb_final[0:5]

[['отец',
  'думал',
  'мать',
  'побоится',
  'ночной',
  'сырости',
  'место',
  'необыкновенно',
  'сухо',
  'никаких',
  'болот',
  'лесу',
  'находилось',
  'поблизости',
  'начиналась',
  'башкирская',
  'степь',
  'влажности',
  'ночного',
  'воздуха',
  'слышно'],
 ['готовилось',
  'новое',
  'зрелище',
  'отложили',
  'лошадей',
  'хотели',
  'спутать',
  'пустить',
  'поле',
  'степные',
  'травы',
  'погорели',
  'солнца',
  'завяли',
  'послали',
  'деревню',
  'свежим',
  'сеном',
  'овсом',
  'всякими',
  'съестными',
  'припасами'],
 ['небо',
  'сверкало',
  'звездами',
  'воздух',
  'наполнен',
  'благовонием',
  'засыхающих',
  'степных',
  'трав',
  'речка',
  'журчала',
  'овраге',
  'костер',
  'пылал',
  'ярко',
  'освещал',
  'наших',
  'людей',
  'которые',
  'сидели',
  'около',
  'котла',
  'горячей',
  'кашицей',
  'хлебали',
  'весело',
  'разговаривали',
  'собою',
  'лошади',
  'припущенные',
  'овсу',
  'также',
  'освещены',
  'одной',
  'стороны',
  'пол

In [92]:
finder2 = BigramCollocationFinder.from_documents(words_sent_orb_final)
# создание списка биграм по токенизированным предложениям

In [97]:
# finder2.score_ngrams(bigram_measures.raw_freq)
# вывести биграмы по частотности

[(('степные', 'кулики'), 0.0013919268244183733),
 (('степных', 'куликов'), 0.0013919268244183733),
 (('оренбургской', 'губернии'), 0.0011930801352157488),
 (('степной', 'кулик'), 0.0011930801352157488),
 (('степной', 'дичи'), 0.000994233446013124),
 (('степные', 'места'), 0.000994233446013124),
 (('это', 'время'), 0.000994233446013124),
 (('дичь', '1'), 0.0007953867568104991),
 (('кречетка', 'степная'), 0.0007953867568104991),
 (('кроншнеп', 'степной'), 0.0007953867568104991),
 (('степная', 'пиголица'), 0.0007953867568104991),
 (('степная', 'полевая'), 0.0007953867568104991),
 (('4', 'кроншнеп'), 0.0005965400676078744),
 (('5', 'кречетка'), 0.0005965400676078744),
 (('9', 'коростель'), 0.0005965400676078744),
 (('алексей', 'степаныч'), 0.0005965400676078744),
 (('болотною', 'водяною'), 0.0005965400676078744),
 (('большими', 'стаями'), 0.0005965400676078744),
 (('всеми', 'породами'), 0.0005965400676078744),
 (('вся', 'степная'), 0.0005965400676078744),
 (('выводит', 'детей'), 0.00059654

In [70]:
bigrams_orb_scored = finder2.score_ngrams(bigram_measures.raw_freq)
# создаем список частот

In [62]:
# bigrams_orb_scored[:10]

[(('—', 'это'), 0.0004730866274179983),
 (('»', '—'), 0.00042052144659377626),
 (('киргизской', 'степи'), 0.00042052144659377626),
 (('это', 'время'), 0.00042052144659377626),
 (('степные', 'кулики'), 0.00036795626576955427),
 (('степных', 'куликов'), 0.00036795626576955427),
 (('—', 'степь'), 0.00036795626576955427),
 (('оренбургской', 'губернии'), 0.0003153910849453322),
 (('степи', 'широкой'), 0.0003153910849453322),
 (('степи', '—'), 0.0003153910849453322)]

In [48]:
# выбрать все биграмы с производными "степь"
# steppe_orb_bigrams_scored = [] # создаем список нужных биграм
# for i in bigrams_orb_scored:
#     for j in i:
#         if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
#             for k in j:
#                 result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
#                 if not result == None: # если есть, добавляем в конечный список
#                     steppe_orb_bigrams_scored.append(i)
#         else: # если проверяемый элемент float, ничего не делаем
#             pass
            
# steppe_orb_bigrams_scored[:10]

[(('киргизской', 'степи'), 0.00042052144659377626),
 (('степные', 'кулики'), 0.00036795626576955427),
 (('степных', 'куликов'), 0.00036795626576955427),
 (('—', 'степь'), 0.00036795626576955427),
 (('степи', 'широкой'), 0.0003153910849453322),
 (('степи', '—'), 0.0003153910849453322),
 (('степной', 'кулик'), 0.0003153910849453322),
 (('«', 'степи'), 0.0002628259041211102),
 (('степной', 'дичи'), 0.0002628259041211102),
 (('степные', 'места'), 0.0002628259041211102)]

In [71]:
# создать словарь с биграммами со словом "степь"
steppe_orb_bigrams_scored_d = dict() # задаем словарь
for i in bigrams_orb_scored:
    for j in i:
        if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
            for k in j:
                result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
                if not result == None: # если есть, добавляем в словарь: кортеж - ключ, число - значение
                    steppe_orb_bigrams_scored_d[i[0]] = i[1]
        else: # если проверяемый элемент float, ничего не делаем
            pass
steppe_orb_bigrams_scored_d

{('киргизский', 'степь'): 0.0009987384356602187,
 ('степной', 'дорога'): 0.0007884777123633305,
 ('степной', 'кулик'): 0.0007884777123633305,
 ('степной', 'место'): 0.0006307821698906644,
 ('вольный', 'степь'): 0.0005782169890664423,
 ('степной', 'простор'): 0.0005256518082422204,
 ('степь', '—'): 0.0005256518082422204,
 ('—', 'степь'): 0.0004730866274179983,
 ('река', 'степь'): 0.00042052144659377626,
 ('степной', 'полевой'): 0.00042052144659377626,
 ('жить', 'степь'): 0.00036795626576955427,
 ('степной', 'дичь'): 0.00036795626576955427,
 ('степной', 'куликов'): 0.00036795626576955427,
 ('степной', 'река'): 0.00036795626576955427,
 ('степной', 'речка'): 0.00036795626576955427,
 ('степь', 'который'): 0.00036795626576955427,
 ('далеко', 'степь'): 0.0003153910849453322,
 ('далёкий', 'степной'): 0.0003153910849453322,
 ('далёкий', 'степь'): 0.0003153910849453322,
 ('смотреть', 'степь'): 0.0003153910849453322,
 ('степной', 'ветер'): 0.0003153910849453322,
 ('степной', 'пиголица'): 0.000315

In [56]:
# dict1 = dict(steppe_orb_bigrams_scored) # создать словарь из списка биграмм

In [72]:
df1 = pd.DataFrame.from_dict (steppe_orb_bigrams_scored_d, orient='index'). reset_index()
df1.columns = ['биграмма', 'количество']
df1

,биграмма,количество
0,"(киргизский, степь)",0.000999
1,"(степной, дорога)",0.000788
2,"(степной, кулик)",0.000788
3,"(степной, место)",0.000631
4,"(вольный, степь)",0.000578
...,...,...
1340,"(юговосточный, степь)",0.000053
1341,"(я, степной)",0.000053
1342,"(явление, степной)",0.000053
1343,"(ягодник, степной)",0.000053


In [73]:
df1.to_csv('bigrams_lem_raw_freq.csv', encoding = "utf-8") # файл для лемматизированных слс

In [93]:
bigrams_orb_scored2 = finder2.score_ngrams(bigram_measures.likelihood_ratio)

In [149]:
# display(bigrams_orb_scored2)

[(('оренбургской', 'губернии'), 75.56975376971161),
 (('степных', 'куликов'), 62.02965192127886),
 (('степные', 'кулики'), 60.777359418696044),
 (('степной', 'кулик'), 52.73119721476844),
 (('алексей', 'степаныч'), 50.54439490885526),
 (('болотною', 'водяною'), 46.045713751904785),
 (('большими', 'стаями'), 46.045713751904785),
 (('крайней', 'мере'), 46.045713751904785),
 (('дичь', '1'), 44.813096869335006),
 (('кречетка', 'степная'), 42.09630765411333),
 (('степная', 'пиголица'), 42.09630765411333),
 (('выводит', 'детей'), 39.95938309832755),
 (('степная', 'полевая'), 39.46694513084915),
 (('всеми', 'породами'), 37.7291418527405),
 (('это', 'время'), 35.55722010317572),
 (('5', 'кречетка'), 35.498900844912704),
 (('выводят', 'детей'), 35.462692783671294),
 (('имел', 'случай'), 35.31852153023083),
 (('малого', 'рода'), 35.31852153023083),
 (('морская', 'ласточка'), 35.31852153023083),
 (('перья', 'крыльях'), 35.31852153023083),
 (('строгом', 'смысле'), 35.31852153023083),
 (('тронутой'

In [51]:
# steppe_orb_bigrams_scored2 = [] # создаем список нужных биграм
# for i in bigrams_orb_scored2:
#     for j in i:
#         if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
#             for k in j:
#                 result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
#                 if not result == None: # если есть, добавляем в конечный список
#                     steppe_orb_bigrams_scored2.append(i)
#         else: # если проверяемый элемент float, ничего не делаем
#             pass
            
# steppe_orb_bigrams_scored2[:10]

[(('степные', 'кулики'), 70.1087661543745),
 (('степных', 'куликов'), 66.12004457534951),
 (('степной', 'кулик'), 54.683097790034104),
 (('кречетка', 'степная'), 49.107598910925944),
 (('степная', 'пиголица'), 49.107598910925944),
 (('киргизской', 'степи'), 48.35268790357746),
 (('степная', 'полевая'), 46.47555744871006),
 (('степных', 'просторов'), 40.81341969779245),
 (('степном', 'сырту'), 39.482833198839714),
 (('степного', 'хищника'), 38.23495760626266)]

In [52]:
# steppe_orb_bigrams_scored2

[(('степные', 'кулики'), 70.1087661543745),
 (('степных', 'куликов'), 66.12004457534951),
 (('степной', 'кулик'), 54.683097790034104),
 (('кречетка', 'степная'), 49.107598910925944),
 (('степная', 'пиголица'), 49.107598910925944),
 (('киргизской', 'степи'), 48.35268790357746),
 (('степная', 'полевая'), 46.47555744871006),
 (('степных', 'просторов'), 40.81341969779245),
 (('степном', 'сырту'), 39.482833198839714),
 (('степного', 'хищника'), 38.23495760626266),
 (('степные', 'места'), 36.142000142116494),
 (('степные', 'пожары'), 35.9568858642379),
 (('степной', 'дичи'), 35.24439549641781),
 (('степною', 'лесною'), 35.09585004091554),
 (('кроншнеп', 'степной'), 31.648752904795494),
 (('степи', 'широкой'), 31.384191565122997),
 (('водяною', 'степною'), 31.276975330554123),
 (('просторы', 'степей'), 30.307716282590317),
 (('степных', 'речек'), 29.817586643683285),
 (('открытой', 'степи'), 27.734926330730143),
 (('степная', 'птица'), 27.570345691882522),
 (('степными', 'дорогами'), 26.39182

In [94]:
steppe_orb_bigrams_scored2_d = dict() # задаем словарь
for i in bigrams_orb_scored2:
    for j in i:
        if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
            for k in j:
                result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
                if not result == None: # если есть, добавляем в словарь: кортеж - ключ, число - значение
                    steppe_orb_bigrams_scored2_d[i[0]] = i[1]
        else: # если проверяемый элемент float, ничего не делаем
            pass
steppe_orb_bigrams_scored2_d

{('киргизский', 'степь'): 93.43673949640748,
 ('степной', 'кулик'): 84.53471355422158,
 ('степной', 'дорога'): 50.96703847544749,
 ('степной', 'простор'): 49.564919949845844,
 ('вольный', 'степь'): 44.31086455327335,
 ('степной', 'куликов'): 42.838449029308805,
 ('степной', 'пиголица'): 40.01544662904752,
 ('степь', 'степь'): 39.846411708903155,
 ('степной', 'полевой'): 36.938973292307544,
 ('степной', 'место'): 32.740486012728894,
 ('степной', 'речка'): 28.723149277525273,
 ('степной', 'сырт'): 28.542774222376558,
 ('кречёткий', 'степной'): 27.63566307319471,
 ('степной', 'дичь'): 24.3472860245934,
 ('воспитані', 'степныхъ'): 21.70686067175958,
 ('даа', 'степьматушка'): 21.70686067175958,
 ('степной…', 'прохладить'): 21.70686067175958,
 ('степныхъ', 'ярмаркахъ'): 21.70686067175958,
 ('степной', 'гулебщик'): 19.980560337712827,
 ('степной', 'сенокос'): 19.980560337712827,
 ('степной', 'хищник'): 19.980560337712827,
 ('жить', 'степь'): 19.78314484260664,
 ('айда', 'степь'): 19.775341147

In [95]:
df2 = pd.DataFrame.from_dict (steppe_orb_bigrams_scored2_d, orient='index'). reset_index()
df2.columns = ['биграмма', 'количество']
df2

,биграмма,количество
0,"(киргизский, степь)",93.436739
1,"(степной, кулик)",84.534714
2,"(степной, дорога)",50.967038
3,"(степной, простор)",49.564920
4,"(вольный, степь)",44.310865
...,...,...
1340,"(степь, кулик)",0.001550
1341,"(солнце, степь)",0.000868
1342,"(степь, место)",0.000339
1343,"(—, степной)",0.000037


In [96]:
df2.to_csv('bigrams_lem_l_ratio.csv', encoding = "utf-8") # файл для лемматизированных слс

In [97]:
bigrams_orb_scored3 = finder2.score_ngrams(bigram_measures.pmi)

In [54]:
# steppe_orb_bigrams_scored3 = [] # создаем список нужных биграм
# for i in bigrams_orb_scored3:
#     for j in i:
#         if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
#             for k in j:
#                 result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
#                 if not result == None: # если есть, добавляем в конесный список
#                     steppe_orb_bigrams_scored3.append(i)
#         else: # если проверяемый элемент float, ничего не делаем
#             pass
            
# steppe_orb_bigrams_scored3[:10]

[(('ваш', 'степной…'), 14.215532999745657),
 (('воспитаніе', 'степныхъ'), 14.215532999745657),
 (('даа', 'степьматушка'), 14.215532999745657),
 (('степной…', 'прохладит'), 14.215532999745657),
 (('степныхъ', 'ярмаркахъ'), 14.215532999745657),
 (('степняка', 'бураном'), 14.215532999745657),
 (('степняки', 'третьего'), 14.215532999745657),
 (('степняком', 'степнягой'), 14.215532999745657),
 (('степняком', 'степнягой'), 14.215532999745657),
 (('степняцкая', 'крепкая'), 14.215532999745657)]

In [98]:
steppe_orb_bigrams_scored3_d = dict() # задаем словарь
for i in bigrams_orb_scored3:
    for j in i:
        if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
            for k in j:
                result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
                if not result == None: # если есть, добавляем в словарь: кортеж - ключ, число - значение
                    steppe_orb_bigrams_scored3_d[i[0]] = i[1]
        else: # если проверяемый элемент float, ничего не делаем
            pass
steppe_orb_bigrams_scored3_d

{('воспитані', 'степныхъ'): 14.215532999745657,
 ('даа', 'степьматушка'): 14.215532999745657,
 ('степной…', 'прохладить'): 14.215532999745657,
 ('степныхъ', 'ярмаркахъ'): 14.215532999745657,
 ('ваш', 'степной…'): 12.6305704990245,
 ('сырой', 'степям…'): 12.6305704990245,
 ('ходкий', 'постепном'): 12.215532999745657,
 ('степи…', 'величаво'): 11.893604904858295,
 ('степи…', 'скандал'): 11.893604904858295,
 ('степняцкий', 'крепкий'): 11.215532999745657,
 ('темнеть', 'степь…'): 11.215532999745657,
 ('степям…', 'василистый'): 11.045607998303344,
 ('солончаковый', 'степи…'): 10.893604904858295,
 ('степняк', 'степняга'): 10.893604904858295,
 ('степь…', 'могила'): 10.893604904858295,
 ('богатый', 'лесостепь'): 10.756101381108358,
 ('незнакомый', 'степняк'): 9.893604904858293,
 ('степняк', 'третий'): 9.893604904858293,
 ('постепном', 'держаться'): 9.756101381108358,
 ('степьматушка', 'говорить'): 9.756101381108358,
 ('страх', 'степняк'): 9.571676809970931,
 ('конь', 'постепном'): 9.515093281604

In [99]:
df3 = pd.DataFrame.from_dict (steppe_orb_bigrams_scored3_d, orient='index'). reset_index()
df3.columns = ['биграмма', 'количество']
df3

,биграмма,количество
0,"(воспитані, степныхъ)",14.215533
1,"(даа, степьматушка)",14.215533
2,"(степной…, прохладить)",14.215533
3,"(степныхъ, ярмаркахъ)",14.215533
4,"(ваш, степной…)",12.630570
...,...,...
1340,"(степь, время)",-0.975912
1341,"(свой, степной)",-1.082959
1342,"(степь, свой)",-2.202188
1343,"(степь, степной)",-3.592287


In [100]:
df3.to_csv('bigrams_lem_pmi.csv', encoding = "utf-8") # файл для лемматизированных слс

# СДЕЛАТЬ ТЕ ЖЕ МЕТРИКИ ДЛЯ ВТОРОЙ ЧАСТИ ТЕКСТОВ

In [14]:
target_sent_others = [] #список предложений со словом степь и его производными
for i in texts_others:
    sentence = sent_tokenize(i)
    for j in sentence:
        result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', j)
        if not result == None:
            target_sent_others.append(j) # finding sentences with a target word and its derivatives and adding target sentences into the list 
# target_sent_orb[0:20]

In [104]:
target_sent_others[0:20]

['она с жаром, с страстью, с слезами, как степная чайка, вилась над детьми своими.',
 'со степи понеслось звонкое ржание жеребенка; красные полосы ясно сверкнули на небе.',
 'а между тем степь уже давно приняла их всех в свои зеленые объятия, и высокая трава, обступивши, скрыла их, и только козачьи черные шапки одни мелькали между ее колосьями.',
 'солнце выглянуло давно на расчищенном небе и живительным, теплотворным светом своим облило степь.',
 'степь чем далее, тем становилась прекраснее.',
 'вон она перевернулась крылами и блеснула перед солнцем… черт вас возьми, степи, как вы хороши!..',
 'вечером вся степь совершенно переменялась.',
 'все пестрое пространство ее охватывалось последним ярким отблеском солнца и постепенно темнело, так что видно было, как тень перебегала по нем, и она становилась темно-зеленою; испарения подымались гуще, каждый цветок, каждая травка испускала амбру, и вся степь курилась благовонием.',
 'пестрые суслики выпалзывали из нор своих, становились на задни

In [105]:
target_sent_others_p = [] #  список предложений без пунктуации
for i in target_sent_others:
    i = i.translate(str.maketrans("", "", string.punctuation)) # удаление пунктуации
    target_sent_others_p.append(i)

In [106]:
sent_others_words = [] # токенизация предложений со словом степь
for i in target_sent_others_p:
    word = word_tokenize(i)
    sent_others_words.append(word)
# sent_orb_words[0:5]

In [115]:
# Очистка списка токенов от стоп-слов и лемматизация
words_sent_others_final = []
for i in sent_others_words:
    w = []
    for j in i:
        if not j in stopwords.words('russian'):
            j = morph.parse(j)[0].normal_form # строка не нужна, если не лемматизировать
            w.append(j)
    words_sent_others_final.append(w)

In [116]:
finder3 = BigramCollocationFinder.from_documents(words_sent_others_final)

In [117]:
bigrams_others_scored = finder3.score_ngrams(bigram_measures.raw_freq)
steppe_others_bigrams_scored_d = dict() # задаем словарь
for i in bigrams_others_scored:
    for j in i:
        if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
            for k in j:
                result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
                if not result == None: # если есть, добавляем в словарь: кортеж - ключ, число - значение
                    steppe_others_bigrams_scored_d[i[0]] = i[1]
        else: # если проверяемый элемент float, ничего не делаем
            pass
# steppe_others_bigrams_scored_d
df4 = pd.DataFrame.from_dict (steppe_others_bigrams_scored_d, orient='index'). reset_index()
df4.columns = ['биграмма', 'мера']
df4

,биграмма,мера
0,"(степь, —)",0.001173
1,"(степь, хутор)",0.000652
2,"(весь, степь)",0.000522
3,"(выехать, степь)",0.000522
4,"(далеко, степь)",0.000522
...,...,...
688,"(—за, степняк)",0.000130
689,"(—мало, степь)",0.000130
690,"(—обыкновенный, степной)",0.000130
691,"(…степь, ветер)",0.000130


In [118]:
df4.to_csv('bigrams_others_lem_raw_freq.csv', encoding = "utf-8")

In [119]:
bigrams_others_scored2 = finder3.score_ngrams(bigram_measures.likelihood_ratio)
steppe_others_bigrams_scored2_d = dict() # задаем словарь
for i in bigrams_others_scored2:
    for j in i:
        if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
            for k in j:
                result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
                if not result == None: # если есть, добавляем в словарь: кортеж - ключ, число - значение
                    steppe_others_bigrams_scored2_d[i[0]] = i[1]
        else: # если проверяемый элемент float, ничего не делаем
            pass
# steppe_others_bigrams_scored_d
df5 = pd.DataFrame.from_dict (steppe_others_bigrams_scored2_d, orient='index'). reset_index()
df5.columns = ['биграмма', 'мера']
df5

,биграмма,мера
0,"(степной, гребень)",26.522633
1,"(братстепняк, правдуматка)",19.890013
2,"(нетронутая…, степнячоокий)",19.890013
3,"(нянькистепнячок, суметь)",19.890013
4,"(постепном, принимать)",19.890013
...,...,...
688,"(—, степной)",0.005241
689,"(степь, дорога)",0.001507
690,"(один, степь)",0.000083
691,"(степь, один)",0.000083


In [120]:
df5.to_csv('bigrams_others_lem_l_ratio.csv', encoding = "utf-8")

In [121]:
bigrams_others_scored3 = finder3.score_ngrams(bigram_measures.pmi)
steppe_others_bigrams_scored3_d = dict() # задаем словарь
for i in bigrams_others_scored3:
    for j in i:
        if type(j) == tuple: # объект в исходном списке частот = ((строка, строка) float), проверяем первый элемент, должен быть кортежем из строк
            for k in j:
                result = re.search(r'[С,с]теп[ьиян].{0,2}|[С,с]тепей', k) # определяем, есть ли искомое слово в кортеже
                if not result == None: # если есть, добавляем в словарь: кортеж - ключ, число - значение
                    steppe_others_bigrams_scored3_d[i[0]] = i[1]
        else: # если проверяемый элемент float, ничего не делаем
            pass
# steppe_others_bigrams_scored_d
df6 = pd.DataFrame.from_dict (steppe_others_bigrams_scored3_d, orient='index'). reset_index()
df6.columns = ['биграмма', 'мера']
df6

,биграмма,мера
0,"(братстепняк, правдуматка)",12.905011
1,"(нетронутая…, степнячоокий)",12.905011
2,"(нянькистепнячок, суметь)",12.905011
3,"(постепном, принимать)",12.905011
4,"(пышно, постепном)",12.905011
...,...,...
688,"(—, степь)",-0.367425
689,"(григорий, степь)",-0.402190
690,"(степь, трава)",-0.842763
691,"(трава, степь)",-0.842763


In [122]:
df6.to_csv('bigrams_others_lem_pmi.csv', encoding = "utf-8")

In [15]:
# Создание таблицы предложений из второй части текстов
# sents1 = pd.DataFrame(target_sent_others)
# sents1.columns = ['Sentence']
# sents1

,Sentence
0,"она с жаром, с страстью, с слезами, как степна..."
1,со степи понеслось звонкое ржание жеребенка; к...
2,а между тем степь уже давно приняла их всех в ...
3,солнце выглянуло давно на расчищенном небе и ж...
4,"степь чем далее, тем становилась прекраснее."
...,...
445,"xviii ранней весною, когда сойдет снег и подсо..."
446,"потоками струится подгоняемый ветром огонь, жа..."
447,"как выжженная палами степь, черна стала жизнь ..."
448,"но сам он все еще судорожно цеплялся за землю,..."


In [20]:
# target_sent_all = target_sent_orb + target_sent_others # Объединение списков предложений

In [24]:
# создание датафрейма из предложений
# sents_fict = pd.DataFrame(target_sent_all)
# sents_fict.columns = ['Sentence']
# sents_fict

,Sentence
0,"отец думал, что мать побоится ночной сырости; ..."
1,для меня опять готовилось новое зрелище; отлож...
2,"небо сверкало звездами, воздух был наполнен бл..."
3,"я видел, будто сквозь сон, как он садился, как..."
4,"степь, то есть безлесная и волнообразная беско..."
...,...
1389,"xviii ранней весною, когда сойдет снег и подсо..."
1390,"потоками струится подгоняемый ветром огонь, жа..."
1391,"как выжженная палами степь, черна стала жизнь ..."
1392,"но сам он все еще судорожно цеплялся за землю,..."


In [25]:
# sents_fict.to_csv('sentences_fict.csv', encoding = "utf-8") # запись предложений из ХТ в файл

In [11]:
# prog = re.compile("[А-Яа-я]+")
# t1 = prog.findall(text.lower())
# t1[0:500]

['п',
 'тр',
 'николаевич',
 'краснов',
 'новомир',
 'в',
 'принципе',
 'рассеянно',
 'сказал',
 'приятель',
 'псина',
 'то',
 'неплохая',
 'хотя',
 'вздрючить',
 'бы',
 'за',
 'проказу',
 'не',
 'мешало',
 'вздрючить',
 'а',
 'спросил',
 'он',
 'глаза',
 'переводя',
 'удостоив',
 'наконец',
 'взглядом',
 'поодаль',
 'благоразумно',
 'сидевшую',
 'собаку',
 'или',
 'малость',
 'погодить',
 'юрок',
 'понимающе',
 'где',
 'тебя',
 'вздрючить',
 'мол',
 'при',
 'такой',
 'то',
 'жаре',
 'и',
 'лени',
 'поглядел',
 'на',
 'него',
 'и',
 'даже',
 'хвостом',
 'не',
 'шевельнул',
 'на',
 'пустые',
 'эти',
 'речи',
 'но',
 'чего',
 'уж',
 'точно',
 'он',
 'не',
 'понимал',
 'так',
 'это',
 'интеллигентского',
 'перекочевавшего',
 'уже',
 'и',
 'к',
 'работягам',
 'и',
 'даже',
 'в',
 'блатную',
 'феню',
 'словца',
 'паразита',
 'гущину',
 'и',
 'самому',
 'однажды',
 'услышать',
 'довелось',
 'угрожающее',
 'в',
 'свой',
 'адрес',
 'ну',
 'т',
 'ты',
 'в',
 'принципе',
 'и',
 'было',
 'это',
 

In [61]:
# t2 = []
# for i in t1:
#     if not i in stopwords.words('russian'):
#         i = morph.parse(i)[0].normal_form
#         t2.append(i)
# t2

['п',
 'тр',
 'николаевич',
 'краснов',
 'новомиро',
 'принцип',
 'рассеянно',
 'сказать',
 'приятель',
 'псиный',
 'неплохой',
 'хотя',
 'вздрючить',
 'проказа',
 'мешать',
 'вздрючить',
 'спросить',
 'глаз',
 'переводить',
 'удостоить',
 'взгляд',
 'поодаль',
 'благоразумно',
 'сидеть',
 'собака',
 'малость',
 'погодить',
 'юркий',
 'понимающе',
 'вздрючить',
 'мол',
 'жара',
 'лень',
 'поглядеть',
 'хвост',
 'шевельнуть',
 'пустой',
 'речь',
 'точно',
 'понимать',
 'это',
 'интеллигентский',
 'перекочевать',
 'работяга',
 'блатной',
 'феня',
 'словцо',
 'паразит',
 'гущина',
 'сам',
 'однажды',
 'услышать',
 'довестись',
 'угрожать',
 'свой',
 'адрес',
 'так',
 'принцип',
 'это',
 'плохой',
 'казаться',
 'натура',
 'прогресс',
 'вовсю',
 'наличествовать',
 'юркий',
 'мочь',
 'понять',
 'беспринципный',
 'собака',
 'встречать',
 'пока',
 'приходиться',
 'маленький',
 'среднее',
 'собачий',
 'размер',
 'примесь',
 'какой',
 'болоночий',
 'кровь',
 'зарасти',
 'нос',
 'длинный',
 'шерс

In [76]:
# t3 = nltk.FreqDist(t2)
# print(t3)
# # t3.most_common(50)

<FreqDist with 27724 samples and 178698 outcomes>


In [77]:
# # длина частотного списка
# nltk.FreqDist.B(t2)

178698

In [95]:
# t3.most_common(50)

[('это', 2248),
 ('свой', 1320),
 ('сказать', 1226),
 ('знать', 999),
 ('сам', 911),
 ('дело', 828),
 ('какой', 806),
 ('весь', 767),
 ('человек', 766),
 ('такой', 685),
 ('говорить', 669),
 ('глаз', 619),
 ('один', 588),
 ('вс', 582),
 ('стать', 547),
 ('тот', 535),
 ('всякий', 518),
 ('наш', 513),
 ('быть', 503),
 ('другой', 485),
 ('мочь', 470),
 ('хотеть', 467),
 ('рука', 453),
 ('думать', 429),
 ('который', 424),
 ('кто', 392),
 ('этот', 389),
 ('время', 387),
 ('жизнь', 379),
 ('мы', 361),
 ('день', 354),
 ('видеть', 349),
 ('самый', 332),
 ('вроде', 329),
 ('первый', 327),
 ('жить', 327),
 ('хотя', 319),
 ('лишь', 319),
 ('глядеть', 317),
 ('понимать', 316),
 ('голова', 313),
 ('ты', 310),
 ('понять', 305),
 ('казаться', 300),
 ('лицо', 296),
 ('дать', 290),
 ('идти', 285),
 ('дом', 266),
 ('м', 264),
 ('никак', 261)]

In [81]:
# # вывести частоту степь
# t3.freq('степь')

0.00030218581069737767

In [99]:
# # вывести абсолютную частоту слова
# for i in t3:
#     if i == 'степь':
#         print(i, t3[i])

степь 54
